In [15]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import log_loss, accuracy_score
import optuna
import lightgbm as lgb
import sklearn
import numpy as np
import matplotlib.pyplot as plt

In [5]:
# read the data from /input/train.csv
data = pd.read_csv("train.csv")

In [6]:
X = data.drop(['Class', 'id'], axis=1)
y = data['Class']

In [7]:
X.head()

,Mean_Integrated,SD,EK,Skewness,Mean_DMSNR_Curve,SD_DMSNR_Curve,EK_DMSNR_Curve,Skewness_DMSNR_Curve
0,133.171875,59.716081,0.043133,-0.703383,54.917224,70.084438,0.749798,-0.649512
1,87.093750,36.257973,0.435469,2.266057,3.417224,21.865069,7.039330,52.686251
2,112.640625,39.818393,0.379639,0.922306,2.730769,15.689690,8.193471,85.649785
3,120.679688,45.918448,-0.098490,0.011775,2.696488,20.954662,8.183874,70.332899
4,134.070312,57.720107,-0.107772,-0.573335,1.107860,11.255051,16.107748,308.753765


# Preprocessing with PolynomialFeatures!!!

In [8]:
# preprocess the data by adding interaction features
#from sklearn.preprocessing import PolynomialFeatures
#poly = PolynomialFeatures(2, interaction_only=True, include_bias=False)
#X = poly.fit_transform(X)

In [9]:
type(X)

pandas.core.frame.DataFrame

# Train, Val, Test Split

In [10]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.1, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(
    X_train, y_train, test_size=0.11, random_state=42)

In [11]:
X

,Mean_Integrated,SD,EK,Skewness,Mean_DMSNR_Curve,SD_DMSNR_Curve,EK_DMSNR_Curve,Skewness_DMSNR_Curve
0,133.171875,59.716081,0.043133,-0.703383,54.917224,70.084438,0.749798,-0.649512
1,87.093750,36.257973,0.435469,2.266057,3.417224,21.865069,7.039330,52.686251
2,112.640625,39.818393,0.379639,0.922306,2.730769,15.689690,8.193471,85.649785
3,120.679688,45.918448,-0.098490,0.011775,2.696488,20.954662,8.183874,70.332899
4,134.070312,57.720107,-0.107772,-0.573335,1.107860,11.255051,16.107748,308.753765
...,...,...,...,...,...,...,...,...
117559,132.843750,56.748838,-0.060070,-0.554084,4.054348,27.844144,6.564423,44.442664
117560,112.578125,52.539271,0.179580,-0.306961,1.637960,15.331913,11.384718,142.535470
117561,119.757812,49.980013,0.064402,-0.270822,3.877926,19.788559,6.959740,56.367789
117562,105.789062,46.986595,0.441426,0.372466,2.097826,17.170612,9.442445,99.074539


In [12]:
X_train.shape

(94168, 8)

# Parameter search with Optuna

In [13]:
def objective(trial): 
    params = {
    'task': 'train', 
    'boosting_type':'gbdt',
    'objective':'binary',
    'metric':'binary_logloss', 
    'verbose':-10000000,
    'seed':42,
    'max_bin':128,
    'n_estimators':128,
    # 'n_estimators':trial.suggest_int("n_estimators", 800, 1200),
    'learning_rate':0.08,
    'feature_fraction':1.0,
    'bagging_fraction':1.0,
   # 'bagging_freq':trial.suggest_categorical("bagging_freq", [1]),
    'max_depth':trial.suggest_int("max_depth", 5, 12), # decrease this in the next round
    'num_leaves':trial.suggest_int("num_leaves",8, 32),
    'min_data_in_leaf':200,
    'min_gain_to_split':1.0,
}
    
    model = lgb.LGBMClassifier(**params)
    model.fit(X_train, y_train, eval_set=[(X_val, y_val)], early_stopping_rounds=100)
    y_val_pred = model.predict_proba(X_val)
    logloss = log_loss(y_val, y_val_pred)
    return logloss

In [17]:
study = optuna.create_study(direction='minimize');
study.optimize(objective, n_trials=10);
#optuna.visualization.plot_optimization_history(study)


[I 2023-03-13 15:51:49,935] A new study created in memory with name: no-name-68545e48-9bd2-4a69-aa6f-e55aad46b501
c:\Users\Ece\anaconda3\lib\site-packages\lightgbm\sklearn.py:726: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "


[LightGBM] [Warning] feature_fraction is set=1.0, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=1.0
[LightGBM] [Warning] min_data_in_leaf is set=200, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=200
[LightGBM] [Warning] min_gain_to_split is set=1.0, min_split_gain=0.0 will be ignored. Current value: min_gain_to_split=1.0
[LightGBM] [Warning] bagging_fraction is set=1.0, subsample=1.0 will be ignored. Current value: bagging_fraction=1.0
[1]	valid_0's binary_logloss: 0.246449
[2]	valid_0's binary_logloss: 0.212494
[3]	valid_0's binary_logloss: 0.18836
[4]	valid_0's binary_logloss: 0.169523
[5]	valid_0's binary_logloss: 0.154069
[6]	valid_0's binary_logloss: 0.141098
[7]	valid_0's binary_logloss: 0.129997
[8]	valid_0's binary_logloss: 0.120353
[9]	valid_0's binary_logloss: 0.111919
[10]	valid_0's binary_logloss: 0.10439
[11]	valid_0's binary_logloss: 0.0976797
[12]	valid_0's binary_logloss: 0.0917019
[13]	valid_0's binary_logloss: 0.0863629

[I 2023-03-13 15:51:50,408] Trial 0 finished with value: 0.03427285365910371 and parameters: {'max_depth': 11, 'num_leaves': 21}. Best is trial 0 with value: 0.03427285365910371.



[93]	valid_0's binary_logloss: 0.0343708
[94]	valid_0's binary_logloss: 0.034314
[95]	valid_0's binary_logloss: 0.0343208
[96]	valid_0's binary_logloss: 0.0342865
[97]	valid_0's binary_logloss: 0.0342729
[98]	valid_0's binary_logloss: 0.0342973
[99]	valid_0's binary_logloss: 0.0343109
[100]	valid_0's binary_logloss: 0.0343007
[101]	valid_0's binary_logloss: 0.0343036
[102]	valid_0's binary_logloss: 0.0343011
[103]	valid_0's binary_logloss: 0.0342869
[104]	valid_0's binary_logloss: 0.03431
[105]	valid_0's binary_logloss: 0.0343158
[106]	valid_0's binary_logloss: 0.0343065
[107]	valid_0's binary_logloss: 0.0343196
[108]	valid_0's binary_logloss: 0.0343319
[109]	valid_0's binary_logloss: 0.0343323
[110]	valid_0's binary_logloss: 0.0343323
[111]	valid_0's binary_logloss: 0.0343323
[112]	valid_0's binary_logloss: 0.0343323
[113]	valid_0's binary_logloss: 0.0343323
[114]	valid_0's binary_logloss: 0.0343323
[115]	valid_0's binary_logloss: 0.0343323
[116]	valid_0's binary_logloss: 0.0343323
[

c:\Users\Ece\anaconda3\lib\site-packages\lightgbm\sklearn.py:726: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "


...

[1]	valid_0's binary_logloss: 0.246449
[2]	valid_0's binary_logloss: 0.212494
[3]	valid_0's binary_logloss: 0.18836
[4]	valid_0's binary_logloss: 0.169523
[5]	valid_0's binary_logloss: 0.154069
[6]	valid_0's binary_logloss: 0.141098
[7]	valid_0's binary_logloss: 0.129997
[8]	valid_0's binary_logloss: 0.120353
[9]	valid_0's binary_logloss: 0.111919
[10]	valid_0's binary_logloss: 0.10439
[11]	valid_0's binary_logloss: 0.0976797
[12]	valid_0's binary_logloss: 0.0917019
[13]	valid_0's binary_logloss: 0.0863629
[14]	valid_0's binary_logloss: 0.0815257
[15]	valid_0's binary_logloss: 0.0772091
[16]	valid_0's binary_logloss: 0.0733193
[17]	valid_0's binary_logloss: 0.0697887
[18]	valid_0's binary_logloss: 0.0666331
[19]	valid_0's binary_logloss: 0.0637304
[20]	valid_0's binary_logloss: 0.0611072
[21]	valid_0's binary_logloss: 0.0587462
[22]	valid_0's binary_logloss: 0.0565292
[23]	valid_0's binary_logloss: 0.0545757
[24]	valid_0's binary_logloss: 0.0527582
[25]	valid_0's binary_logloss: 0.0511

[I 2023-03-13 15:51:54,360] Trial 8 finished with value: 0.03422394065196606 and parameters: {'max_depth': 12, 'num_leaves': 21}. Best is trial 1 with value: 0.03386998075382093.
c:\Users\Ece\anaconda3\lib\site-packages\lightgbm\sklearn.py:726: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "


[LightGBM] [Warning] feature_fraction is set=1.0, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=1.0
[LightGBM] [Warning] min_data_in_leaf is set=200, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=200
[LightGBM] [Warning] min_gain_to_split is set=1.0, min_split_gain=0.0 will be ignored. Current value: min_gain_to_split=1.0
[LightGBM] [Warning] bagging_fraction is set=1.0, subsample=1.0 will be ignored. Current value: bagging_fraction=1.0
[1]	valid_0's binary_logloss: 0.24647
[2]	valid_0's binary_logloss: 0.212519
[3]	valid_0's binary_logloss: 0.18839
[4]	valid_0's binary_logloss: 0.169556
[5]	valid_0's binary_logloss: 0.154103
[6]	valid_0's binary_logloss: 0.141138
[7]	valid_0's binary_logloss: 0.130052
[8]	valid_0's binary_logloss: 0.12036
[9]	valid_0's binary_logloss: 0.111933
[10]	valid_0's binary_logloss: 0.104404
[11]	valid_0's binary_logloss: 0.0977014
[12]	valid_0's binary_logloss: 0.0917499
[13]	valid_0's binary_logloss: 0.0864016


[I 2023-03-13 15:51:54,862] Trial 9 finished with value: 0.03396449453089007 and parameters: {'max_depth': 11, 'num_leaves': 18}. Best is trial 1 with value: 0.03386998075382093.



[96]	valid_0's binary_logloss: 0.0340596
[97]	valid_0's binary_logloss: 0.0340545
[98]	valid_0's binary_logloss: 0.0340721
[99]	valid_0's binary_logloss: 0.0340617
[100]	valid_0's binary_logloss: 0.034062
[101]	valid_0's binary_logloss: 0.0340406
[102]	valid_0's binary_logloss: 0.0340027
[103]	valid_0's binary_logloss: 0.0339866
[104]	valid_0's binary_logloss: 0.0340068
[105]	valid_0's binary_logloss: 0.0340092
[106]	valid_0's binary_logloss: 0.0340308
[107]	valid_0's binary_logloss: 0.0340298
[108]	valid_0's binary_logloss: 0.0340334
[109]	valid_0's binary_logloss: 0.0340276
[110]	valid_0's binary_logloss: 0.034031
[111]	valid_0's binary_logloss: 0.0340202
[112]	valid_0's binary_logloss: 0.033977
[113]	valid_0's binary_logloss: 0.0339796
[114]	valid_0's binary_logloss: 0.0339645
[115]	valid_0's binary_logloss: 0.0339645
[116]	valid_0's binary_logloss: 0.0339645
[117]	valid_0's binary_logloss: 0.0339645
[118]	valid_0's binary_logloss: 0.0339645
[119]	valid_0's binary_logloss: 0.033964

In [18]:
study.best_trial.params

{'max_depth': 11, 'num_leaves': 12}

In [19]:
params = {
    'task': 'train', 
    'boosting_type':'gbdt',
    'objective':'binary',
    'metric':'binary_logloss', 
    'verbose':-10000000,
    'seed':42,
    'max_bin':128,
    'n_estimators':128,
    # 'n_estimators':trial.suggest_int("n_estimators", 800, 1200),
    'learning_rate':0.08,
    'feature_fraction':1.0,
    'bagging_fraction':1.0,
   # 'bagging_freq':trial.suggest_categorical("bagging_freq", [1]),
    'min_data_in_leaf':200,
    'min_gain_to_split':1.0,
}
    

In [20]:
params.update(study.best_trial.params)

In [22]:
tuned_model = lgb.LGBMClassifier(**params)
tuned_model.fit(X_train, y_train, 
        eval_set=[(X_val, y_val)],
        eval_metric=['binary_logloss'],
        early_stopping_rounds=100) 

c:\Users\Ece\anaconda3\lib\site-packages\lightgbm\sklearn.py:726: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "


[LightGBM] [Warning] feature_fraction is set=1.0, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=1.0
[LightGBM] [Warning] min_data_in_leaf is set=200, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=200
[LightGBM] [Warning] min_gain_to_split is set=1.0, min_split_gain=0.0 will be ignored. Current value: min_gain_to_split=1.0
[LightGBM] [Warning] bagging_fraction is set=1.0, subsample=1.0 will be ignored. Current value: bagging_fraction=1.0
[1]	valid_0's binary_logloss: 0.246505
[2]	valid_0's binary_logloss: 0.212586
[3]	valid_0's binary_logloss: 0.188484
[4]	valid_0's binary_logloss: 0.169664
[5]	valid_0's binary_logloss: 0.154295
[6]	valid_0's binary_logloss: 0.141353
[7]	valid_0's binary_logloss: 0.130239
[8]	valid_0's binary_logloss: 0.12061
[9]	valid_0's binary_logloss: 0.112134
[10]	valid_0's binary_logloss: 0.104625
[11]	valid_0's binary_logloss: 0.0979358
[12]	valid_0's binary_logloss: 0.09198
[13]	valid_0's binary_logloss: 0.086669
[

LGBMClassifier(bagging_fraction=1.0, feature_fraction=1.0, learning_rate=0.08,
               max_bin=128, max_depth=11, metric='binary_logloss',
               min_data_in_leaf=200, min_gain_to_split=1.0, n_estimators=128,
               num_leaves=12, objective='binary', seed=42, task='train',
               verbose=-10000000)

In [ ]:
X_test.shape

(11757, 36)

In [23]:
preds = tuned_model.predict_proba(X_test)
testloss = log_loss(y_test, preds)
testloss

0.030570310091813074

In [24]:
train_preds = tuned_model.predict_proba(X_train)
trainloss = log_loss(y_train, train_preds)
trainloss

0.026942563847874208

In [25]:
val_preds = tuned_model.predict_proba(X_val)
valloss = log_loss(y_val, val_preds)
valloss

0.03386998075382093

# Submission Pipeline

In [ ]:
# read the real test data from test.csv
test_data = pd.read_csv("test.csv")

In [ ]:
X_testtest = test_data.drop(['id'], axis=1)
ID_test = test_data.id

In [ ]:
X_testtest = poly.fit_transform(X_testtest)

In [ ]:
preds = tuned_model.predict_proba(X_testtest)

In [ ]:
preds

array([[9.99515200e-01, 4.84800360e-04],
       [9.99230283e-01, 7.69716583e-04],
       [9.99826850e-01, 1.73149779e-04],
       ...,
       [9.99832612e-01, 1.67387506e-04],
       [9.35309601e-01, 6.46903993e-02],
       [1.61411435e-02, 9.83858856e-01]])

In [ ]:
preds=  preds[:,1]

In [ ]:
preds

array([4.84800360e-04, 7.69716583e-04, 1.73149779e-04, ...,
       1.67387506e-04, 6.46903993e-02, 9.83858856e-01])

In [ ]:
preds.shape

(78377,)

In [ ]:
# Save test predictions to file
submission = pd.DataFrame({'id': ID_test,
                       'Class': preds})
submission.to_csv('submission_optuna_lgbm.csv', index=False)